In [56]:
import pandas as pd

seeds = {
    "uniform-cifar10": [0],
    "uniform-cifar20": [0],
    "clcifar10": [0, 2, 10],
    "clcifar20": [0, 2, 10],
    "clcifar10-iid": [0, 2, 10],
    "clcifar20-iid": [0, 2, 10],
}

algos = ["fwd-u", "fwd-r", "ure-ga-u", "ure-ga-r", "scl-nl", "scl-exp", "l-w", "l-uw"]

columns = ['selected by URE', 'URE', 'selected by SCEL', 'SCEL', 'selected by val_acc', 'valid_acc', 'best_ure', 'best_scel', 'best_val_acc']


## 1. Synthetic Uniform CL v.s. CLCIFAR

In [54]:
# "selected by val_acc", "best_val_acc" for each algorithm
exp1_result = {}

for dataset in ["uniform-cifar10", "clcifar10", "uniform-cifar20", "clcifar20"]:
    df = []
    for seed in seeds[dataset]:
        df.append(pd.read_csv(f"{seed}/{dataset}.csv"))

    def reformat(row):
        row['acc'] = f"{row['mean'].round(2)}({row['std'].round(2)})"
        row = row.drop(['mean', 'std'])
        return row
    
    dataset_df = pd.DataFrame()
    for algo in algos:
        result = []
        for i in range(len(seeds[dataset])):
            if df[i].loc[df[i]['algo'] == algo].shape[0] == 0:
                break
            result.append(df[i].loc[df[i]['algo'] == algo])
        if not result:
            dataset_df[algo] = "-"
            continue
        res = pd.DataFrame()
        res['mean'] = pd.concat(result, axis=0).mean(axis=0, numeric_only=True)
        res['std'] = pd.concat(result, axis=0).std(axis=0, numeric_only=True)
        res = res.apply(reformat, axis=1).transpose()

        dataset_df[algo] = res.loc['acc']
    
    
    dataset_df = dataset_df.transpose()
    dataset_df.drop(['selected by URE', 'URE', 'selected by SCEL', 'SCEL', 'valid_acc', 'best_ure', 'best_scel'], axis=1, inplace=True)
    dataset_df.rename(columns={"selected by val_acc": "valid_acc", "best_val_acc": "valid_acc (ES)"}, inplace=True)
    
    exp1_result[dataset] = dataset_df
    
exp1_result = pd.concat(exp1_result, axis=1)
exp1_result


uniform-cifar10                   clcifar10                 \
               valid_acc valid_acc (ES)    valid_acc valid_acc (ES)   
fwd-u         49.12(nan)     49.28(nan)  34.09(1.16)    36.83(1.17)   
fwd-r                  -              -  28.88(0.65)     38.9(1.57)   
ure-ga-u      39.59(nan)     39.59(nan)  34.59(0.76)    36.39(0.67)   
ure-ga-r               -              -   28.7(1.39)    30.94(1.66)   
scl-nl        48.63(nan)      48.5(nan)   33.8(0.52)    37.81(2.12)   
scl-exp       48.17(nan)     47.64(nan)  34.59(0.72)    36.96(0.18)   
l-w           25.84(nan)     45.62(nan)  28.04(0.38)    34.55(2.05)   
l-uw          31.66(nan)     47.14(nan)  30.63(1.87)    35.13(1.56)   

         uniform-cifar20                   clcifar20                 
               valid_acc valid_acc (ES)    valid_acc valid_acc (ES)  
fwd-u         20.32(nan)     19.87(nan)   7.47(0.37)     8.27(0.77)  
fwd-r                  -              -  16.14(1.11)    20.31(0.25)  
ure-ga-u      15.48(nan)     15.48(nan)   7.59(0.36)    10.06(0.72)  
ure-ga-r               -              -    5.24(0.2)     5.46(0.28)  
scl-nl        18.92(nan)     19.56(nan)   7.58(0.66)     8.53(0.29)  
scl-exp       18.78(nan)     20.63(nan)   7.55(0.51)     8.11(0.71)  
l-w           12.69(nan)     17.55(nan)    7.08(1.1)     8.74(0.42)  
l-uw          14.66(nan)     17.14(nan)   7.36(0.33)     8.71(0.31)

1. fwd-r suffers from overfitting on both clcifar10 and clcifar20. It is not the case when T is uniform.
2. ure-ga-u turn out to be a robust choice in clcifar20.

## 2. Validation Objectives

In [52]:
# "selected by val_acc", "best_val_acc" for each algorithm
exp2_result = {}

for dataset in ["clcifar10", "clcifar20"]:
    df = []
    for seed in seeds[dataset]:
        df.append(pd.read_csv(f"{seed}/{dataset}.csv"))

    def reformat(row):
        row['acc'] = f"{row['mean'].round(2)}({row['std'].round(2)})"
        row = row.drop(['mean', 'std'])
        return row
    
    dataset_df = pd.DataFrame()
    for algo in algos:
        result = []
        for i in range(len(seeds[dataset])):
            if df[i].loc[df[i]['algo'] == algo].shape[0] == 0:
                print(i, algo, dataset)
                break
            result.append(df[i].loc[df[i]['algo'] == algo])
        if not result:
            dataset_df[algo] = "-"
            continue
        res = pd.DataFrame()
        res['mean'] = pd.concat(result, axis=0).mean(axis=0, numeric_only=True)
        res['std'] = pd.concat(result, axis=0).std(axis=0, numeric_only=True)
        res = res.apply(reformat, axis=1).transpose()

        dataset_df[algo] = res.loc['acc']
    
    
    dataset_df = dataset_df.transpose()
    dataset_df.drop(['URE', 'SCEL', 'valid_acc', 'best_val_acc', 'valid_acc'], axis=1, inplace=True)
    dataset_df = dataset_df.reindex(columns=["selected by URE", "best_ure", "selected by SCEL", "best_scel", "selected by val_acc"])
    dataset_df = dataset_df.rename(columns={"selected by URE": "URE", "best_ure": "URE(ES)", "selected by SCEL":"SCEL", "best_scel":"SCEL(ES)", "selected by val_acc":"valid acc"})
    
    exp2_result[dataset] = dataset_df
    
exp2_result = pd.concat(exp2_result, axis=1)
exp2_result

clcifar10                                                      \
                  URE      URE(ES)         SCEL     SCEL(ES)    valid acc   
fwd-u     32.54(0.79)  33.28(0.49)  33.02(1.67)   32.9(1.62)  34.09(1.16)   
fwd-r     28.88(0.65)  27.57(1.09)  19.13(0.75)  19.13(0.75)  28.88(0.65)   
ure-ga-u  31.34(1.08)  31.51(0.86)  34.59(0.76)  33.93(0.97)  34.59(0.76)   
ure-ga-r   26.61(0.8)  26.14(1.01)  28.18(1.84)   29.6(2.23)   28.7(1.39)   
scl-nl    32.49(0.66)   32.9(0.07)  33.32(1.54)  33.72(1.54)   33.8(0.52)   
scl-exp   33.15(1.08)   33.13(0.7)   31.26(2.4)   31.44(2.5)  34.59(0.72)   
l-w       20.71(0.82)  20.72(0.96)  20.71(0.82)  21.06(1.64)  28.04(0.38)   
l-uw      22.97(0.13)  22.64(0.37)  22.97(0.13)  22.71(0.44)  30.63(1.87)   

            clcifar20                                                    
                  URE      URE(ES)        SCEL    SCEL(ES)    valid acc  
fwd-u      7.33(0.37)   7.29(0.48)  7.33(0.62)  6.47(1.34)   7.47(0.37)  
fwd-r     10.74(0.76)  10.54(0.87)   9.8(0.25)   9.8(0.25)  16.14(1.11)  
ure-ga-u   7.28(0.18)   7.69(0.02)  7.85(0.16)   7.69(0.2)   7.59(0.36)  
ure-ga-r    5.2(0.25)   4.86(0.04)  5.36(0.18)  5.25(0.16)    5.24(0.2)  
scl-nl      7.12(0.8)   7.28(1.03)   6.5(0.68)  6.52(1.38)   7.58(0.66)  
scl-exp    6.85(0.88)   7.05(0.79)  6.53(0.92)  6.55(1.36)   7.55(0.51)  
l-w        5.77(0.38)   5.65(0.28)  6.03(0.51)  6.34(0.54)    7.08(1.1)  
l-uw        5.9(0.21)   5.78(0.77)   5.9(0.21)  6.49(0.36)   7.36(0.33)

## 3. Feature Independence

In [58]:
exp3_result = {}

for dataset in ["clcifar10", "clcifar10-iid", "clcifar20", "clcifar20-iid"]:
    df = []
    for seed in seeds[dataset]:
        df.append(pd.read_csv(f"{seed}/{dataset}.csv"))

    def reformat(row):
        row['acc'] = f"{row['mean'].round(2)}({row['std'].round(2)})"
        row = row.drop(['mean', 'std'])
        return row
    
    dataset_df = pd.DataFrame()
    for algo in algos:
        result = []
        for i in range(len(seeds[dataset])):
            if df[i].loc[df[i]['algo'] == algo].shape[0] == 0:
                break
            result.append(df[i].loc[df[i]['algo'] == algo])
        if not result:
            dataset_df[algo] = "-"
            continue
        res = pd.DataFrame()
        res['mean'] = pd.concat(result, axis=0).mean(axis=0, numeric_only=True)
        res['std'] = pd.concat(result, axis=0).std(axis=0, numeric_only=True)
        res = res.apply(reformat, axis=1).transpose()

        dataset_df[algo] = res.loc['acc']
    
    
    dataset_df = dataset_df.transpose()
    dataset_df.drop(['selected by URE', 'URE', 'selected by SCEL', 'SCEL', 'valid_acc', 'best_ure', 'best_scel', "best_val_acc"], axis=1, inplace=True)
    dataset_df.rename(columns={"selected by val_acc": "valid_acc"}, inplace=True)
    
    exp3_result[dataset] = dataset_df
    
exp3_result = pd.concat(exp3_result, axis=1)
exp3_result


,clcifar10,clcifar10-iid,clcifar20,clcifar20-iid
,valid_acc,valid_acc,valid_acc,valid_acc
fwd-u,34.09(1.16),32.99(2.15),7.47(0.37),6.83(0.33)
fwd-r,28.88(0.65),28.52(0.76),16.14(1.11),12.61(1.16)
ure-ga-u,34.59(0.76),31.56(0.77),7.59(0.36),7.22(0.3)
ure-ga-r,28.7(1.39),29.43(1.06),5.24(0.2),7.02(2.75)
scl-nl,33.8(0.52),33.13(2.21),7.58(0.66),7.3(0.35)
scl-exp,34.59(0.72),32.62(1.15),7.55(0.51),7.17(0.72)
l-w,28.04(0.38),25.54(0.3),7.08(1.1),6.58(0.69)
l-uw,30.63(1.87),27.1(0.5),7.36(0.33),6.53(0.28)


## 4. Different Data Cleaning Rate

## 5. Learning with Multiple CL